In [42]:
import os
import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains 
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from bs4 import BeautifulSoup
import requests
from datetime import datetime, timedelta
import re

page = 1
year = 2024
BASE_URL = "https://www.dailypioneer.com/"
country = "bangladesh"
initial_url = f'{BASE_URL}searchlist.php?section=&adv={country}&yr={year}&page={page}'

all_news_links = []

#IN THIS WEBSITE, THE ARTICLES ARE SEPARATED BY YEAR. FOR EACH YEAR, THERE ARE LIMITED NUMBERS OF PAGES.

#Collecting news articles till the first 2 pages of 2023. There's only 1 page for 2024 so far. 

page = 1
year = 2024 

while year!= 2022:       

        initial_url = f'{BASE_URL}searchlist.php?section=&adv={country}&yr={year}&page={page}'
              
        
        response = requests.get(initial_url)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        #CHECKING IF THE PAGE FOR THE PARTICULAR YEAR EXISTS
        
        news_first_part_div = soup.find('div', class_ = 'highLightedNews')
        
        print(f'Year {year}, Page {page} - ')

        if news_first_part_div:

            page_exists = True
            print('Page Available')

        else:

            page_exists = False
            print('Page Not Available')
            
        
        #THE NEWS FOR EACH PAGE IS AVAILABLE IN 3 DIFFERENT FORMS. ONLY ONE NEWS DISPLAYED AS A LARGE TILE, 
        #A SECOND STRING OF NEWS DISPLAYED BESIDE THE FIRST NEWS IN COLUMNS
        #A THIRD STRING OF NEWS DISPLAYED BELOW THE FIRST TWO NEWS SECTIONS IN THE FORM OF COLUMNS.

        if page_exists:

                first_news = news_first_part_div.find('div', class_ = 'col-12 col-md-8').find('a')

                all_news_links.append(first_news.get('href'))

                second_col_news_lists = news_first_part_div.find('ul', class_ = 'list-unstyled')

                news_lists = second_col_news_lists.find_all('li')

                for each_news_list in news_lists:

                    all_news_links.append(each_news_list.find('a').get('href'))

                news_second_part_div = soup.find('div', class_ = 'innerNewsList')

                second_part_news_lists = news_second_part_div.find_all('div', class_ = 'row newsWrap no-gutters')

                for each_news_list in second_part_news_lists:

                    all_news_links.append(each_news_list.find('a').get('href'))
                    
                page+=1
        else:
            
            year -=1 
            page = 1
            
        
        
        if year == 2023 and page == 3:
            
            break

            
#SCRAPING THE NEWS LINKS

counter = 0
data_list = []

#SCRAPING THE FIRST 20 LINKS
for link in all_news_links[:20]:

        url = f'{BASE_URL}{link}'

        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        #TITLE

        title_tag = soup.find('h2', {'itemprop':'headline'})
        title = title_tag.text if title_tag else 'Title not found'
        title_translation = 'None'

        #Author

        author_tag = soup.find('span',{'itemprop':'author'})
        author = author_tag.text if author_tag else 'Author not found'

        #Date Info
        #There's no information about time. Only DATE and DAY.

        date_info= soup.find('span',{'itemprop':'dateModified'})
        date_data = date_info.get('content') if date_info else 'Date data not found'

        if date_data != 'Date data not found':

            source_localtime = datetime.strptime(date_data, '%A, %d %B %Y')
            bangladesh_localtime = source_localtime

        #NO CONTENT SUMMARIES AVAILABLE IN THIS WEBSITE

        content_summary = 'None'
        summary_translation = 'None'

        #Content

        content = []

        content_main_div = soup.find('div', {'itemprop':'articleBody'})

        if content_main_div:

                all_paras = content_main_div.find_all('p')

                for para in all_paras:
                    content.append(para.text)

                full_content = ''.join(content)
                full_content = full_content.strip()
                full_content = re.sub('\n|\r',' ',full_content)

        else:
                full_content = 'Content not found'

        content_translation = 'None'
        
        data_dict = {
            "url": url,
            "title": title,
            "content": full_content,
            "content_summary": content_summary,
            "title_translation":title_translation,
            "content_translation":content_translation,
            "summary translation":summary_translation,
            "author": author,
            "country": country,
            'source_localtime': source_localtime,
            'bangladesh_localtime': bangladesh_localtime

        }

        counter+=1


        if (full_content != "Content Not Found" and title != 'Title not found'):

                if data_dict not in data_list:
                        # Adding to data list
                        data_list.append(data_dict)
                        print(f'Link {counter} added')
        else:
                print(f'Link {counter}')
                print('Skipped due to missing info.')


df = pd.DataFrame(data_list)
df.head()


csv_filename = f"{country}_The_Pioneer.csv"

# Checking if the CSV file already exists
if os.path.exists(csv_filename):
    existing_df = pd.read_csv(csv_filename)
    # Merging new and existing dataframe
    df = pd.concat([existing_df, pd.DataFrame(data_list)], ignore_index=True)
    df["bangladesh_localtime"] = pd.to_datetime(df["bangladesh_localtime"])  # Converting the "date" column to datetime
    df = df.drop_duplicates(subset=["title"], keep="first")
    df = df.sort_values(by="date", ascending=False)  # Sorting the date
    df = df.reset_index(drop=True)
    df.to_csv(csv_filename, index=False)  
else:
    # If csv file does not exist, then we create a new CSV file with the scraped data
    df = pd.DataFrame(data_list)
    df = df.sort_values(by="bangladesh_localtime", ascending=False)
    df = df.drop_duplicates(subset=["title"], keep="first")
    df = df.reset_index(drop=True)
    df.to_csv(csv_filename, index=False)


Year 2024, Page 1 - 
Page Available
Year 2024, Page 2 - 
Page Not Available
Year 2023, Page 1 - 
Page Available
Year 2023, Page 2 - 
Page Available
Link 1 added
Link 2 added
Link 3 added
Link 4 added
Link 5 added
Link 6 added
Link 7 added
Link 8 added
Link 9 added
Link 10 added
Link 11 added
Link 12 added
Link 13 added
Link 14 added
Link 15 added
Link 16 added
Link 17 added
Link 18 added
Link 19 added
Link 20 added


,url,title,content,content_summary,title_translation,content_translation,summary translation,author,country,source_localtime,bangladesh_localtime
0,https://www.dailypioneer.com//2024/sports/bang...,Bangladesh pacer Mustafizur hospitalised after...,Experienced left-arm pacer Mustafizur Rahman w...,None,None,None,None,PTI,bangladesh,2024-02-19,2024-02-19
1,https://www.dailypioneer.com//2024/world/three...,Three Bangladeshi women arrested in Tripura,Three Bangladeshi women and three locals were ...,None,None,None,None,PTI,bangladesh,2024-02-12,2024-02-12
2,https://www.dailypioneer.com//2024/world/myanm...,Myanmar forces fighting ethnic group flee in...,n Nearly a hundred members of Myanmar’s Border...,None,None,None,None,AP,bangladesh,2024-02-06,2024-02-06
3,https://www.dailypioneer.com//2024/sports/indi...,India face Nepal after loss to Bangladesh,Smarting from a disappointing 0-1 defeat to Ba...,None,None,None,None,PTI,bangladesh,2024-02-06,2024-02-06
4,https://www.dailypioneer.com//2024/state-editi...,BJP holds march against Bangladeshi and Rohing...,Under the leadership of Lohardaga District Bha...,None,None,None,None,lOHARDAGA,bangladesh,2024-01-21,2024-01-21
